In [2]:
!ls -l data/modelnet40_normal_resampled/points_embeddings/test

total 7193812
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:50 0.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:53 100.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:53 101.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 59695416 Mar 19 19:53 102.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:50 10.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 11.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 12.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 13.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 14.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 15.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 16.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 17.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 18.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 19.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:50 1.pkl
-rw-rw-r-- 1 d.tarasov d.tarasov 71633352 Mar 19 19:51 

In [3]:
?os.setenv

Object `os.setenv` not found.


In [47]:
import os

os.environ['LD_LIBRARY_PATH'] = '/home/d.tarasov/anaconda3_new/lib'

os.environ['LD_LIBRARY_PATH']

'/home/d.tarasov/anaconda3_new/lib'

In [48]:
from glob import glob


In [49]:
from data_utils.ModelNetDataLoader import ModelNetDataLoader
import argparse

parser = argparse.ArgumentParser('Testing')
parser.add_argument('--use_cpu', action='store_true', default=False, help='use cpu mode')
parser.add_argument('--gpu', type=str, default='0', help='specify gpu device')
parser.add_argument('--batch_size', type=int, default=24, help='batch size in training')
parser.add_argument('--num_category', default=40, type=int, choices=[10, 40],  help='training on ModelNet10/40')
parser.add_argument('--num_point', type=int, default=1024, help='Point Number')
parser.add_argument('--log_dir', type=str, default='pointnet2_msg_normals', help='Experiment root')
parser.add_argument('--use_normals', action='store_true', default=False, help='use normals')
parser.add_argument('--use_uniform_sample', action='store_true', default=False, help='use uniform sampiling')
parser.add_argument('--num_votes', type=int, default=3, help='Aggregate classification scores with voting')
parser.add_argument('--split', type=str, default='test', help='train/test dataset split to evaluate')
parser.add_argument('--save_embeddings', action='store_true', default=False, help='save points embeddings for each batch')
args = parser.parse_args([])

test_dataset = ModelNetDataLoader(root='data/modelnet40_normal_resampled', args=args, split='test', process_data=False)

The size of test data is 2468


In [50]:
len(test_dataset)

2468

In [51]:
points_embeddings_sorted = sorted(glob('data/modelnet40_normal_resampled/points_embeddings/test/*.pkl'), key=lambda x: int(x.split('.')[0].split('/')[-1]))

In [55]:
torch.cuda.is_available()

False

In [57]:
?pickle.load

In [56]:
import pickle

with open(points_embeddings_sorted[0], 'rb') as f:
    batch_embeddings = pickle.load(f, map_location=torch.device('cpu'))


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [43]:
# [ кол-во голосов, кол-во слоев, ... (размерности) ]
batch_embeddings[-1][-2]['points'].shape

NameError: name 'batch_embeddings' is not defined

In [44]:
batch_embeddings[:][-2]['points']

NameError: name 'batch_embeddings' is not defined

In [45]:
from torch.utils.data import Dataset
import torch

class PointNet2EmbeddingsDataset(Dataset):
    
    def __init__(self, modelnet_dataset, embedding_level=-1, vote_idx=-1):
    
        self.modelnet_dataset = modelnet_dataset
        
        points_embeddings_files = glob(f'{self.modelnet_dataset.root}/points_embeddings/{self.modelnet_dataset.split}/*.pkl')
        
        self.embeddings_files = sorted(points_embeddings_files, key=lambda x: int(x.split('.')[0].split('/')[-1]))
        
        embedding_file = self.load_embedding_file(0)
        self.votes_num = len(embedding_file)

        self.modelnet_batch_size = embedding_file[-1][-1]['points'].shape[0]
        
        self.embedding_level = embedding_level
        self.vote_idx = vote_idx
        
    def __len__(self):
        return len(self.modelnet_dataset)
    
    def load_embedding_file(self, file_idx):
        filename = self.embeddings_files[file_idx]
    
        
        with open(filename, 'rb') as f:
            batch_embeddings = pickle.load(f)
            
        return batch_embeddings

    def _get_modelnet_label(self, idx):
        return self.modelnet_dataset.classes[self.modelnet_dataset.datapath[ idx ][0]]

    
    def __getitem__(self, idx):
        
        
        label = self._get_modelnet_label(idx)
        all_batch_embedding_from_file = self.load_embedding_file(idx // self.modelnet_batch_size)
        
        
        local_batch_idx = idx % self.modelnet_batch_size

        vote = all_batch_embedding_from_file[self.vote_idx]
        vote_embeddings = vote[self.embedding_level]['points']

        model_embeddings = vote_embeddings[ local_batch_idx, :, : ]
        
        return {
            "label": label,
            "label_name": self.modelnet_dataset.cat[label],
            "embeddings": model_embeddings
        }

    

In [65]:
PointNet2EmbeddingsDataset(test_dataset)[1235]

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [114]:
len(PointNet2EmbeddingsDataset(test_dataset))

2468

In [113]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

for x in tqdm(DataLoader(PointNet2EmbeddingsDataset(test_dataset), batch_size=2, shuffle=False), total=len(test_dataset)//2):

#     print(x['label_names'])
#     break
    pass

KeyboardInterrupt: 

In [61]:
1 //3

0

In [41]:
len(test_dataset), 24 * len(points_embeddings_sorted)

(2468, 2472)

In [25]:
test_dataset.cat

['airplane',
 'bathtub',
 'bed',
 'bench',
 'bookshelf',
 'bottle',
 'bowl',
 'car',
 'chair',
 'cone',
 'cup',
 'curtain',
 'desk',
 'door',
 'dresser',
 'flower_pot',
 'glass_box',
 'guitar',
 'keyboard',
 'lamp',
 'laptop',
 'mantel',
 'monitor',
 'night_stand',
 'person',
 'piano',
 'plant',
 'radio',
 'range_hood',
 'sink',
 'sofa',
 'stairs',
 'stool',
 'table',
 'tent',
 'toilet',
 'tv_stand',
 'vase',
 'wardrobe',
 'xbox']

In [16]:
import transformers

print(transformers.__version__)

model_checkpoint = "t5-small"
print("model_checkpoint", model_checkpoint)


from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

4.18.0
model_checkpoint t5-small


Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/610320 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /home/d.tarasov/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

In [18]:
raw_datasets["train"][0]

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

In [64]:
?model.generate

In [19]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [63]:
torch.full([ 3, 16 ], tokenizer.pad_token_id, dtype=torch.long, device=)

TypeError: full() received an invalid combination of arguments - got (list, int, device=type, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, Number fill_value, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, Number fill_value, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [60]:
tokenizer.pad_token_id

0

In [20]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [79]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is sdfsdf sd fsdf sdfsdf sdfs dfsdf sf another sentence"]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 3, 7, 26, 89, 7, 26, 89, 3, 7, 26, 3, 89, 7, 26, 89, 3, 7, 26, 89, 7, 26, 89, 3, 7, 26, 89, 7, 3, 26, 89, 7, 26, 89, 3, 7, 89, 430, 7142, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [80]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "what is it? "
else:
    prefix = ""

In [89]:
max_input_length = 128
max_target_length = 128
source_field = ""
target_field = "label_name"

embeddings_3d_length = 16

def preprocess_function(examples):
    inputs = [prefix for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

#     model_inputs["extra_embeddings"] = examples["embeddings"]
    # extra_embeddings_positions
    model_inputs['extra_embeddings_positions'] = torch.LongTensor([ len(x) for x in model_inputs['attention_mask'] ])

    
    for seq in model_inputs['input_ids']:
        seq.extend([ tokenizer.pad_token_id ] * embeddings_3d_length)
        
    return model_inputs

In [90]:
preprocess_function(raw_datasets['train'][:20])

{'input_ids': [[125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125, 19, 34, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [125,

In [25]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/611 [00:00<?, ?ba/s]

KeyboardInterrupt: 

In [26]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [27]:

from transformers.models.t5.modeling_t5 import T5Stack
import torch.nn as nn

class T5EncoderWithExtraEmbeddings(T5Stack):
    
    def forward(self, **kwargs):
        
        extra_embeddings = kwargs.get('extra_embeddings') # [ bs, seq_len, embedding_dim ]
        extra_embeddings_positions = kwargs.get('extra_embeddings_positions') # [ bs, seq_len ]
        
        if extra_embeddings ^ extra_embeddings_positions:
            raise Exception('T5EncoderWithExtraEmbeddings expects both extra_embeddings_positions and extra_embeddings to be passed to')
        
        model_out = super().forward(**kwargs)
        
        if extra_embeddings is not None and extra_embeddings_positions is not None:
            for i in extra_embeddings.shape[0]:
                extra_embedding_seq = extra_embeddings[i, :, :]
                extra_embedding_start_pos = extra_embeddings_positions[i]

                model_out.last_hidden_state[i, extra_embedding_start_pos:, :] = extra_embedding_seq
    
        return model_out
    

In [28]:
transformers.__version__

'4.18.0'

In [29]:
import transformers.utils as tutils 

In [33]:
from transformers.models.t5.modeling_t5 import T5ForConditionalGeneration, T5PreTrainedModel

import copy
import math
import os
import warnings
from typing import Optional, Tuple, Union

import torch
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.utils.checkpoint import checkpoint


from typing import Optional, Tuple, Union
import torch
from transformers.modeling_outputs import (
    BaseModelOutput,
    BaseModelOutputWithPastAndCrossAttentions,
    Seq2SeqLMOutput,
    Seq2SeqModelOutput,
)

class T5ForConditionalGenerationExtraEmbeddings(T5ForConditionalGeneration, T5PreTrainedModel):
    
    def __init__(self, config):
        super().__init__(config)

        self.model_dim = config.d_model

        self.shared = nn.Embedding(config.vocab_size, config.d_model)

        encoder_config = copy.deepcopy(config)
        encoder_config.is_decoder = False
        encoder_config.use_cache = False
        encoder_config.is_encoder_decoder = False
        
        # DIFF
        self.encoder = T5EncoderWithExtraEmbeddings(encoder_config, self.shared)

        decoder_config = copy.deepcopy(config)
        decoder_config.is_decoder = True
        decoder_config.is_encoder_decoder = False
        decoder_config.num_layers = config.num_decoder_layers
        self.decoder = T5Stack(decoder_config, self.shared)

        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)

        # Initialize weights and apply final processing
        self.post_init()

        # Model parallel
        self.model_parallel = False
        self.device_map = None
    
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.BoolTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        decoder_head_mask: Optional[torch.FloatTensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        
        # DIFF
        extra_embeddings: Optional[torch.FloatTensor] = None, # [ bs, seq_len, embeddings_dim ]
        extra_embeddings_positions: Optional[torch.LongTensor] = None, # [ bs ]
    ) -> Union[Tuple[torch.FloatTensor], Seq2SeqLMOutput]:
        
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # FutureWarning: head_mask was separated into two input args - head_mask, decoder_head_mask
        if head_mask is not None and decoder_head_mask is None:
            if self.config.num_layers == self.config.num_decoder_layers:
                warnings.warn(__HEAD_MASK_WARNING_MSG, FutureWarning)
                decoder_head_mask = head_mask

        # Encode if needed (training, first prediction pass)
        if encoder_outputs is None:
            # Convert encoder inputs in embeddings if needed
            encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                inputs_embeds=inputs_embeds,
                head_mask=head_mask,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
                extra_embeddings=extra_embeddings,
                extra_embeddings_positions=extra_embeddings_positions,
            )
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
            )

        hidden_states = encoder_outputs[0]

        if self.model_parallel:
            torch.cuda.set_device(self.decoder.first_device)

        if labels is not None and decoder_input_ids is None and decoder_inputs_embeds is None:
            # get decoder inputs from shifting lm labels to the right
            decoder_input_ids = self._shift_right(labels)

        # Set device for model parallelism
        if self.model_parallel:
            torch.cuda.set_device(self.decoder.first_device)
            hidden_states = hidden_states.to(self.decoder.first_device)
            if decoder_input_ids is not None:
                decoder_input_ids = decoder_input_ids.to(self.decoder.first_device)
            if attention_mask is not None:
                attention_mask = attention_mask.to(self.decoder.first_device)
            if decoder_attention_mask is not None:
                decoder_attention_mask = decoder_attention_mask.to(self.decoder.first_device)

        # Decode
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            inputs_embeds=decoder_inputs_embeds,
            past_key_values=past_key_values,
            encoder_hidden_states=hidden_states,
            encoder_attention_mask=attention_mask,
            head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = decoder_outputs[0]

        # Set device for model parallelism
        if self.model_parallel:
            torch.cuda.set_device(self.encoder.first_device)
            self.lm_head = self.lm_head.to(self.encoder.first_device)
            sequence_output = sequence_output.to(self.lm_head.weight.device)

        if self.config.tie_word_embeddings:
            # Rescale output before projecting on vocab
            # See https://github.com/tensorflow/mesh/blob/fa19d69eafc9a482aff0b59ddd96b025c0cb207d/mesh_tensorflow/transformer/transformer.py#L586
            sequence_output = sequence_output * (self.model_dim**-0.5)

        lm_logits = self.lm_head(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(lm_logits.view(-1, lm_logits.size(-1)), labels.view(-1))
            # TODO(thom): Add z_loss https://github.com/tensorflow/mesh/blob/fa19d69eafc9a482aff0b59ddd96b025c0cb207d/mesh_tensorflow/layers.py#L666

        if not return_dict:
            output = (lm_logits,) + decoder_outputs[1:] + encoder_outputs
            return ((loss,) + output) if loss is not None else output

        return Seq2SeqLMOutput(
            loss=loss,
            logits=lm_logits,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions,
        )


In [31]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [34]:
model = T5ForConditionalGenerationExtraEmbeddings.from_pretrained('t5-small')

In [ ]:
model

In [27]:
?model.generate

In [21]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [23]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

/home/d.tarasov/anaconda3_new/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [25]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
        
    # todo compute accuracy not BLEU!
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [26]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

NameError: name 'tokenized_datasets' is not defined

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline

en_fr_translator = pipeline("translation_en_to_ro", model=model, tokenizer=tokenizer, device=0)
en_fr_translator("How old are you?")